<a href="https://colab.research.google.com/github/Nazmul1932/SPL_3/blob/main/spl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import pandas as pd
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import gensim
from gensim.utils import simple_preprocess as SIMPLE_PREPROCESS  
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS as STOP_WORDS

from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import *
import numpy as np
from gensim import corpora,models
import time
import pickle
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


**1. Differentiate Master report and duplicate report**

In [35]:
pd.options.display.max_colwidth = 1000

In [62]:
csv_file = "/content/drive/MyDrive/eclipse_platform.csv"

data = pd.read_csv(csv_file)
# data.head(1)

In [63]:
data = data.drop(columns=['Priority', 'Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'])
# data.head(1)

In [64]:
data = data.dropna(axis=0, subset=['Description'])
# data.head(1)

In [65]:
#Removing the fixed patterns by replacing them with " "
data["Description"]= data["Description"].str.replace("fixed in HEAD", "", case = False)
data["Description"]= data["Description"].str.replace("has been marked as readonly", " ", case = False)

In [66]:
import re
import string

def clean_text_from_description_title(text):

    text = text.replace('\t', '')

    text = re.sub('\w*\d\w*', '', text)
    text = text.lower()

    text = text.translate(str.maketrans('', '', string.punctuation))
    
    return text

step_1 = lambda x: clean_text_from_description_title(x)
step_2 = lambda x: clean_text_from_description_title(x)

data['Description'] = data['Description'].apply(step_1)
data['Title'] = data['Title'].apply(step_2)
data.loc[[0]]

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,usability issue with external editors,setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with an external editor found the modified file on disk and showed the changes the real problem occurs if release actually finds something to release but you dont spot that some resources are missing this is extremely error prone one of my changes didnt made it into build because of this notes eg pm release should do a refresh from local before doing the release moving to vcm km pm comments from jm in related email should not do this for free could have a setting which made it optoinal but...


In [67]:

def removalWord_tokenization_lemmatization(text):
    result = []

    for token in SIMPLE_PREPROCESS(text):
        if token not in STOP_WORDS  and len(token) > 5:
            result.append(WordNetLemmatizer().lemmatize(token, pos='v'))
            
    return result


data['Title'] = data['Title'].map(removalWord_tokenization_lemmatization)
data['Description'] = data['Description'].map(removalWord_tokenization_lemmatization)

data.loc[[0]]

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [68]:
duplicated_reports = data.dropna(axis=0, subset=['Duplicated_issue'])
duplicated_reports.reset_index(drop=True, inplace=True)
duplicated_reports.head(1)

,Issue_id,Duplicated_issue,Title,Description
0,25,28126.0,"[default, connection]","[helpful, notion, default, repository, connection, connect, project, wouldnt, everytime, create, create, default, minimum, have, default, repository, helpful, pulldown, connection, wizard, default]"


In [78]:
master_reports = data[data.isnull().any(axis=1)]
master_reports.reset_index(drop=True, inplace=True)
master_reports.head(1)

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [86]:
m_r = pd.DataFrame()

for i in range(len(master_reports)):
    if(len(master_reports.Description[i])>2):
        m_r = m_r.append(master_reports.loc[[i]])

    

In [87]:
#Creating a dictionary using gensim library
#the dictionary object is used to create a bag of words (BoW) corpus which further used as the input to topic modelling and other models as well

dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 500:
        break


0 actually
1 builders
2 change
3 comment
4 compare
5 complete
6 consider
7 consistency
8 contain
9 default
10 derive
11 document
12 drive
13 editor
14 effect
15 ensure
16 external
17 extremely
18 fail
19 inconsistent
20 internal
21 maintain
22 mandatory
23 miss
24 modify
25 move
26 navigator
27 network
28 occur
29 open
30 optimal
31 problem
32 project
33 refresh
34 relate
35 release
36 resource
37 resources
38 responsible
39 runnable
40 set
41 show
42 summary
43 verify
44 workspace
45 workspaces
46 accomodate
47 agree
48 content
49 critical
50 dependent
51 discuss
52 display
53 doesnt
54 edit
55 editors
56 eventually
57 example
58 expect
59 general
60 graphic
61 map
62 need
63 outliner
64 possible
65 reference
66 remote
67 repositories
68 repository
69 result
70 sensible
71 specialize
72 vanilla
73 write
74 appear
75 arrows
76 attention
77 better
78 clarification
79 clearer
80 correctly
81 delete
82 deletion
83 distinction
84 especially
85 focus
86 indicate
87 indication
88 indicator
8

In [88]:
#Creating BoW using the the dictionary
#it is the corpus that contains the word id and its frequency in each document

bag_of_words_corpus = [dictionary.doc2bow(doc) for doc in master_reports['Description']]
bag_of_words_corpus[0]

[(0, 1),
 (1, 2),
 (2, 5),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 3),
 (14, 1),
 (15, 1),
 (16, 3),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 2),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 2),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 2),
 (32, 3),
 (33, 3),
 (34, 1),
 (35, 12),
 (36, 5),
 (37, 2),
 (38, 1),
 (39, 2),
 (40, 2),
 (41, 1),
 (42, 1),
 (43, 2),
 (44, 2),
 (45, 1)]

In [89]:
#Printing the BoW for single document
bag_of_words_doc = bag_of_words_corpus[0]
for i in range(len(bag_of_words_doc)):
    print("{}. (\"{}\") word seen {} time.".format(i+1, dictionary[bag_of_words_doc[i][0]], bag_of_words_doc[i][1]))
                                               

1. ("actually") word seen 1 time.
2. ("builders") word seen 2 time.
3. ("change") word seen 5 time.
4. ("comment") word seen 1 time.
5. ("compare") word seen 1 time.
6. ("complete") word seen 1 time.
7. ("consider") word seen 1 time.
8. ("consistency") word seen 1 time.
9. ("contain") word seen 1 time.
10. ("default") word seen 1 time.
11. ("derive") word seen 1 time.
12. ("document") word seen 1 time.
13. ("drive") word seen 1 time.
14. ("editor") word seen 3 time.
15. ("effect") word seen 1 time.
16. ("ensure") word seen 1 time.
17. ("external") word seen 3 time.
18. ("extremely") word seen 1 time.
19. ("fail") word seen 1 time.
20. ("inconsistent") word seen 1 time.
21. ("internal") word seen 1 time.
22. ("maintain") word seen 2 time.
23. ("mandatory") word seen 1 time.
24. ("miss") word seen 1 time.
25. ("modify") word seen 1 time.
26. ("move") word seen 1 time.
27. ("navigator") word seen 2 time.
28. ("network") word seen 1 time.
29. ("occur") word seen 1 time.
30. ("open") word s

In [ ]:
corpus = bag_of_words_corpus

#Training the LDA model on the BoW corpus
lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=20, workers=2, iterations=100)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.057*"dialog" + 0.051*"select" + 0.046*"button" + 0.019*"selection" + 0.017*"preference" + 0.016*"change" + 0.013*"preferences" + 0.013*"toolbar" + 0.013*"disable" + 0.013*"enable"
Topic: 1 
Words: 0.026*"method" + 0.018*"change" + 0.018*"return" + 0.017*"action" + 0.014*"support" + 0.012*"create" + 0.012*"provide" + 0.011*"call" + 0.010*"object" + 0.010*"currently"
Topic: 2 
Words: 0.080*"message" + 0.049*"exception" + 0.032*"occur" + 0.024*"follow" + 0.023*"fail" + 0.022*"arguments" + 0.020*"eclipse" + 0.018*"cause" + 0.017*"session" + 0.017*"unable"
Topic: 3 
Words: 0.060*"thread" + 0.049*"launch" + 0.035*"eclipse" + 0.030*"browser" + 0.030*"javathread" + 0.029*"daemon" + 0.025*"threadblocked" + 0.019*"reproducible" + 0.018*"identifier" + 0.018*"lock"
Topic: 4 
Words: 0.097*"bundle" + 0.062*"warn" + 0.061*"miss" + 0.059*"message" + 0.057*"libjvmdylib" + 0.047*"resolve" + 0.041*"orgeclipseosgi" + 0.036*"libsystembdylib" + 0.034*"libclientdylib" + 0.030*"subentry"
To

In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(bag_of_words_corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -6.5547748375582735


In [ ]:
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda_model, texts=data['Description'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.6115455529348007


In [ ]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bag_of_words_corpus, dictionary)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.119108  0.028124       1        1  15.906215
9     -0.120170 -0.056259       2        1  13.442050
5     -0.137067  0.002641       3        1  11.577776
8     -0.096514 -0.109371       4        1  11.494096
0     -0.197826  0.107072       5        1  11.366212
6      0.004014 -0.169710       6        1  10.190739
2      0.153173 -0.104295       7        1   7.701216
7     -0.080034  0.278740       8        1   7.517286
3      0.195918 -0.185295       9        1   6.706652
4      0.397613  0.208354      10        1   4.097759, topic_info=             Term          Freq         Total Category  logprob  loglift
32        project  21983.000000  21983.000000  Default  30.0000  30.0000
567       message  12619.000000  12619.000000  Default  29.0000  29.0000
13         editor  12139.000000  12139.000000  Default  28.0000  28.0000
340        public   9531.000000   9531.000000  Default  27.0000  27.0000
727        import  10516.000000  10516.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
1282       thread    715.751352   6662.709853  Topic10  -4.4328   0.9638
177        active    446.364433   2068.098996  Topic10  -4.9050   1.6615
386   environment    385.571781   1746.682475  Topic10  -5.0514   1.6840
241       version    527.105027   9352.285752  Topic10  -4.7387   0.3188
65      reference    378.902214   2862.889289  Topic10  -5.0688   1.1724

[627 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
210       1  0.983872        ability
210       6  0.015547        ability
3118      6  0.995694      abouthtml
794       1  0.363493         access
794       2  0.036835         access
...     ...       ...            ...
44        9  0.013467      workspace
45        2  0.998148     workspaces
4218      9  0.997602      xulrunner
4211      9  0.998068  xxmaxpermsize
2807      5  0.991420         yellow

[1191 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 10, 6, 9, 1, 7, 3, 8, 4, 5])

In [ ]:
#Creating 10 empty clusters and pushing master reports in each of them based on topic modeling and saving them in individual csv file
import pandas as pd

for c in range(10):
    exec('topic_{} = pd.DataFrame()'.format(c))

    for i in range(len(master_reports)):
        topic=lda_model[dictionary.doc2bow(master_reports.Description[i])]
        topic= np.asarray(topic)
        
        if int(topic[np.argmax(topic[:,1]),0])== c:
            exec('topic_{} = topic_{}.append(master_reports.loc[[i]])'.format(c,c))
            exec('topic_{} = topic_{}.reset_index(drop=True)'.format(c,c))
            exec('topic_{}.to_csv("topic_{}.csv")'.format(c,c))